<h1 align=center>Classificando imagens com redes neurais convolucionais profundas</h1>
<p align=center><img src=https://www.electricalelibrary.com/wp-content/uploads/2018/11/convolutional_neural_network.png></p>


Já analisamos detalhadamente diferentes aspectos da API do *TensorFlow*, você se familiarizou com tensores e funções de decoração e aprendeu a trabalhar com os estimadores do *TensorFlow*. Aqui você aprenderá agora sobre **redes neurais convolucionais (CNNs)** para classificação de imagens. Começaremos discutindo os blocos básicos de construção das CNNs, usando uma abordagem de baixo para cima. Em seguida, mergulharemos mais fundo na arquitetura CNN e exploraremos como implementar CNNs no *TensorFlow*.

### Os blocos de construção das CNNs
CNNs são uma família de modelos que foram originalmente inspirados em como o córtex visual do cérebro humano funciona ao reconhecer objetos. O desenvolvimento das CNNs remonta à década de 1990, quando *Yann LeCun* e seus colegas propuseram uma nova arquitetura RN para classificar dígitos manuscritos de imagens (Handwritten Digit Recognition with a Back-Propagation Network, Y. LeCun e outros, 1989, publicado no Conferência de Sistemas de Processamento de Informações Neurais (NeurIPS).

> ### O córtex visual humano
> A descoberta original de como o córtex visual do nosso cérebro funciona foi feita por David H. Hubel e Torsten Wiesel em 1959, quando inseriram um microeletrodo no córtex visual primário de um gato anestesiado. Então, eles observaram que os neurônios do cérebro respondem de maneira diferente depois de projetar diferentes padrões de luz na frente do gato. Isso acabou levando à descoberta das diferentes camadas do córtex visual. Enquanto a camada primária detecta principalmente bordas e linhas, camadas de ordem superior concentram-se mais na extração de formas e padrões complexos.

Devido ao excelente desempenho das CNNs para tarefas de classificação de imagens, esse tipo específico de RN *feedforward* ganhou muita atenção e levou a enormes melhorias no aprendizado de máquina para visão computacional. Vários anos depois, em 2019, *Yann LeCun* recebeu o prêmio *Turing* (o prêmio de maior prestígio em ciência da computação) por suas contribuições ao campo da Inteligência Artificial (IA), juntamente com outros dois pesquisadores, *Yoshua Bengio e Geoffrey Hinton*.

A seguir, discutiremos o conceito mais amplo de RNs e por que as arquiteturas convolucionais são frequentemente descritas como **camadas de extração de recursos**. Em seguida, aprofundaremos a definição teórica do tipo de operação de convolução que é comumente usada em CNNs e percorreremos exemplos para calcular convoluções em uma e duas dimensões.

### Entendendo as CNNs e as hierarquias de recursos

A extração bem-sucedida de **recursos salientes (relevantes)** é fundamental para o desempenho de qualquer algoritmo de aprendizado de máquina e os modelos tradicionais de aprendizado de máquina dependem de recursos de entrada que podem vir de um especialista de domínio ou são baseados em técnicas computacionais de extração de recursos.

Certos tipos de RNs, como CNNs, são capazes de aprender automaticamente os recursos de dados brutos que são mais úteis para uma tarefa específica. Por esse motivo, é comum considerar as camadas CNN como extratores de recursos: as camadas iniciais (aquelas logo após a camada de entrada) extraem **características de baixo nível** de dados brutos e as camadas posteriores (geralmente, **camadas totalmente conectadas** como em um perceptron multicamada (*MLP*)) usa esses recursos para prever um valor de destino contínuo ou rótulo de classe.

Certos tipos de RNs multicamadas e, em particular, RNs convolucionais profundos (CNNs), constroem a chamada **hierarquia de recursos** combinando os recursos de baixo nível em uma forma de camada para formar recursos de alto nível. Por exemplo, se estamos lidando com imagens, os recursos de baixo nível, como bordas e bolhas, são extraídos das camadas anteriores, que são combinadas para formar recursos de alto nível. Esses recursos de alto nível podem formar formas mais complexas, como os contornos gerais de objetos como prédios, gatos ou cachorros.

Como você pode ver na imagem a seguir, uma CNN calcula mapas de recursos de uma imagem de entrada, onde cada elemento vem de um patch local de pixels na imagem de entrada:

<img src=https://miro.medium.com/max/1000/1*z7hd8FZeI_eodazwIapvAw.png>

Este patch local de pixels é referido como o **campo receptivo local**. As CNNs geralmente têm um desempenho muito bom em tarefas relacionadas à imagem, e isso se deve em grande parte a duas ideias importantes:
* **Conectividade esparsa**: Um único elemento no mapa de recursos é conectado a apenas um pequeno trecho de pixels. (Isto é muito diferente de conectar-se a toda a imagem de entrada como no caso dos perceptrons)
* **Compartilhamento de parâmetros**: Os mesmos pesos são usados ​​para diferentes patches da imagem de entrada.

Como consequência direta dessas duas ideias, substituir um *MLP* convencional totalmente conectado por uma camada de convolução **diminui substancialmente** o número de pesos (parâmetros) na rede e veremos uma melhoria na capacidade de capturar recursos salientes. No contexto de dados de imagem, faz sentido supor que pixels próximos são tipicamente mais relevantes entre si do que pixels distantes.

Normalmente, as CNNs são compostas por várias camadas convolucionais e de subamostragem que são seguidas por uma ou mais camadas totalmente conectadas no final. As camadas totalmente conectadas são essencialmente um *MLP*, onde cada unidade de entrada, $\small i$, está conectada a cada unidade de saída, $\small j$, com peso $\small w_{ij}$.

Observe que as camadas de subamostragem, comumente conhecidas como **camadas de agrupamento** (*pooling layers*), não possuem parâmetros que podem ser aprendidos; por exemplo, não há pesos ou unidades de polarização nas **camadas de agrupamento**. No entanto, ambas as camadas convolucional e totalmente conectada têm pesos e vieses que são otimizados durante o treinamento.

Nas seções a seguir, estudaremos as camadas convolucionais e de *pooling* com mais detalhes e veremos como elas funcionam. Para entender como as operações de convolução funcionam, vamos começar com uma convolução em uma dimensão, que às vezes é usada para trabalhar com certos tipos de dados de sequência, como texto. Depois de discutir as convoluções unidimensionais, trabalharemos com as convoluções bidimensionais típicas que são comumente aplicadas a imagens bidimensionais. 

### Executando convoluções discretas

Uma **convolução discreta** (ou simplesmente convolução) é uma operação fundamental em uma CNN. Portanto, é importante entender como essa operação funciona. Abordaremos a definição matemática e discutiremos alguns dos algoritmos ingênuos para calcular convoluções de tensores unidimensionais (vetores) e tensores bidimensionais (matrizes).

Observe que as fórmulas e descrições nesta seção são apenas para entender como funcionam as operações de convolução nas CNNs. De fato, implementações muito mais eficientes de operações convolucionais já existem em pacotes como o *TensorFlow*, como você verá mais adiante.

### Convoluções discretas em uma dimensão
Vamos começar com algumas definições e notações básicas que vamos usar. Uma convolução discreta para dois vetores, $\small x$ e $\small w$, é denotada por $y = x \times w$ , em que o vetor x é nossa entrada (às vezes chamado de **signal**) e $\small w$ é chamado de **filter** ou **kernel**. Uma convolução discreta é definida matematicamente da seguinte forma:

$$
y = x \times w \to y[i] = \sum^{+ \infty}_{k=-\infty}x[i-k]w[k]
$$

Os colchetes, $[ \: ]$ , são usados ​​para denotar a indexação de elementos vetoriais. O índice, $\small i$, percorre cada elemento do vetor de saída, $\small y$. Há duas coisas estranhas na fórmula anterior que precisamos esclarecer: $−\infty$ a $+\infty$ índices e indexação negativa para $\small x$.
O fato de a soma percorrer índices de $−\infty$ a $+\infty$ parece estranho, principalmente porque em aplicações de aprendizado de máquina, sempre lidamos com vetores de características finitos. Por exemplo, se $\small x$ tem 10 traços com índices 0, 1, 2,…, 8, 9, então os índices $−\infty$∶ −1 e 10 ∶ $+\infty$ estão fora dos limites para $\small x$. Portanto, para calcular corretamente a soma mostrada na fórmula anterior, assume-se que $\small x$ e $\small w$ são preenchidos com zeros. Isso resultará em um vetor de saída, $\small y$, que também tem tamanho infinito, com muitos zeros também. Como isso não é útil em situações práticas, $\small x$ é preenchido apenas com um número finito de zeros.

Esse processo é chamado de **zero-padding** (preenchimento zero) ou simplesmente **padding**. Aqui, o número de zeros preenchidos em cada lado é denotado por $\small p$. Um exemplo de preenchimento de um vetor unidimensional, $\small x$, é mostrado na figura a seguir:

![](https://img-blog.csdnimg.cn/20200905045036227.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0xpbmxpNTIyMzYyMjQy,size_16,color_FFFFFF,t_70)

Vamos supor que a entrada original, $\small x$, e o filtro, $\small w$, tenham `n` e `m` elementos, respectivamente, onde $\small  m \leq n$ . Portanto, o vetor preenchido, $\small x^p$ , tem tamanho $\small n + 2p$. A fórmula prática para calcular uma convolução discreta mudará para o seguinte:

$$
y = x \times w \to y[i] = \sum^{k=m-1}_{k=0}x^{p}[i+m-k]w[k]
$$

Agora que resolvemos o problema do índice infinito, o segundo problema é indexar $\small x$ com $\small i + m – k$. O ponto importante a ser observado aqui é que $\small x$ e $\small w$ são indexados em direções diferentes nessa soma. Calcular a soma com um índice indo na direção inversa é equivalente a calcular a soma com ambos os índices na direção direta depois de inverter um desses vetores, $\small x$ ou $\small w$, depois de serem preenchidos. Então, podemos simplesmente calcular seu produto escalar. Vamos supor que viramos (giramos) o filtro, $\small w$, para obter o filtro girado, $\small w^r$. Então, o produto escalar, $\small x[i:i+m]$. $\small w^r$, é calculado para obter um elemento, $\small y[i]$, onde $\small x[i: i + m]$ é um patch de $\small x$ com tamanho $small m$. Esta operação é repetida como em uma abordagem de janela deslizante para obter todos os elementos de saída. A figura a seguir fornece um exemplo com $\small x = [3 \: 2\: 1\: 7\: 1\: 2\: 5\: 4]$ e $\small w = [\dfrac{1}{2} \: \dfrac{3}{4} \: 1\:  \dfrac{1}{4}]$ para que os três primeiros elementos de saída sejam calculados:

![](https://img-blog.csdnimg.cn/20200906095751465.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0xpbmxpNTIyMzYyMjQy,size_16,color_FFFFFF,t_70)

Você pode ver no exemplo anterior que o tamanho do preenchimento é zero ($\small p = 0$). Observe que o filtro girado, $\small w^r$ , é deslocado em duas células cada vez que deslocamos (**shift**). Esse deslocamento é outro hiperparâmetro de uma convolução, o **stride** (passo), $\small s$. Neste exemplo, o passo é dois, $\small s = 2$. Observe que o passo deve ser um número positivo menor que o tamanho do vetor de entrada. Falaremos mais sobre *padding* e *strides* posteriomente.

### Preenchimento de entradas para controlar o tamanho dos mapas de recursos de saída
Até agora, usamos apenas *zero-padding* em convoluções para calcular vetores de saída de tamanho finito. Tecnicamente, o preenchimento pode ser aplicado com qualquer $\small p \geq 0$ . Dependendo da escolha de $\small p$, as células limítrofes podem ser tratadas de forma diferente das células localizadas no meio de $\small x$.

Agora, considere um exemplo onde $\small n = 5$ e $\small m = 3$. Então, com $\small p=0$, $\small x[0]$ é usado apenas no cálculo de um elemento de saída (por exemplo, $\small y[0]$), enquanto $\small x[1]$ é usado no cálculo de dois elementos de saída (por exemplo, $\small y[0]$ e $\small y[1]$). Então, você pode ver que esse tratamento diferente dos elementos de $\small x$ pode colocar artificialmente mais ênfase no elemento do meio, $\small x[2]$, já que ele apareceu na maioria dos cálculos. Podemos evitar esse problema se escolhermos $\small p = 2$, caso em que cada elemento de $\small x$ estará envolvido no cálculo de três elementos de $\small y$.

Além disso, o tamanho da saída, $\small y$, também depende da escolha da estratégia de preenchimento que usamos.

Existem três modos de preenchimento que são comumente usados na prática: *full* (completo), *same* (igual) e *valid* (válido):
* No modo *full*, o parâmetro de *padding*, $\small p$, é definido como $\small p = m – 1$. O preenchimento *full* aumenta as dimensões da saída; assim, <u>raramente é usado em arquiteturas CNN</u>.
* O *Same* preenchimento geralmente é usado para garantir que o vetor de saída tenha o mesmo tamanho que o vetor de entrada, $\small x$. Nesse caso, o parâmetro de preenchimento, $\small p$, é calculado de acordo com o tamanho do filtro, juntamente com o requisito de que o tamanho de entrada e o tamanho de saída sejam os mesmos.
* Finalmente, computar uma convolução no modo *valid* refere-se ao caso em que $\small p = 0$ (sem *padding*).

![](imagens\padding.PNG)

